Here we go! So I still need to figure out some details on how to use these notebooks and what a good structure would be.  Let's keep this simple and look at the finished product when needed. 

## This is interesting 

Action Plan:
1. Create a folder structure for images (already done)
2. Create a validation and sample set
3. Finetune and train model
4. Generate Predictions 
5. Validate Predictions
6. Submit Predictions to Kaggle

The folder structure as you created it in AWS:
utils/
    vgg16.py
    utils.py
lesson1/
    valid/
    results/
    redux.ipynb
    data/
        redux/
            train/
                cat.437.jpg
                dog.9924.jpg
                cat.1029.jpg
                dog.4374.jpg
            test/
                231.jpg
                325.jpg
                1235.jpg
                9923.jpg

In [3]:
%cd lesson1

/home/ubuntu/courses/deeplearning1/nbs/lesson1


In [4]:
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs/lesson1'

In [8]:
#Here we create a homebase for directories
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/redux'

In [6]:
#Allow relative imports to directories above Lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

#import modules
from utils import *
from vgg16 import Vgg16

#Instantiate plotting tool
#In Jupyter notebooks you will need to do this before doing any plotting 
%matplotlib inline

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


Action Plan
1. Create a Validation and Sample sets
2. Rearrange image files into their respective directories
3. Finetune and train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

# Create a validation set and sample

In [ ]:
#Create directories (This is already done)
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p sample/unknown

In [7]:
%cd %DATA_HOME_DIR/train

[Errno 2] No such file or directory: '%DATA_HOME_DIR/train'
/home/ubuntu/courses/deeplearning1/nbs/lesson1


In [ ]:
# glob will find any file, in this case with a .jpg
g = glob('*.jpg')
# np.random.permutation will randomly permutate a sequence (from docs ;)) permutation- a way things can be done
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_HOME_DIR+'/valid/' + shuf[i])

In [ ]:
from shutil import copyfile

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
#from docs: copyfile(source, destination)
for i in range of(200): copyfile(shuf[i], DATA_HOME_DIR+'sample/train/' + shuf[i])

In [ ]:
%cd $DATA_HOME_DIR/valid

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
#from docs: copyfile(source, destination)
for i in range of(50): copyfile(shuf[i], DATA_HOME_DIR+'sample/valid/' + shuf[i])

# Rearrange Image Files into their respective directories

In [ ]:
#Divide cat and dog images into separate directories

%cd $DATA_HOME_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
#Create a single unknown class for test set
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

# Finetuning and Training 

In [14]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample'
test_path = DATA_HOME_DIR + '/test' #We use all the test data
results_path = DATA_HOME_DIR + '/results/'
train_path = path + '/train/'
valid_path = path + '/valid'

/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux


In [15]:
# import Vgg16 helper class
vgg = Vgg16()

In [16]:
#Set constants. You can experiment with number of epochs to improve the model
batch_size = 64
no_of_epochs = 3

In [17]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size= batch_size)
val_batches = vgg.get_batches(valid_path, batch_size= batch_size*2)
vgg.finetune(batches)

#Not sure if we set this for all fits
vgg.model.optimizer.lr = 0.01

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [19]:
#Notice we are passing in the validation dataset to the fit() method
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print "Running epoch: %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' %epoch
    vgg.model.save_weights(results_path + latest_weights_filename)
print "Completed %s fit operations" % no_of_epochs

Running epoch: 0
Epoch 1/1
23000/23000 [==============================] - 646s - loss: 0.3599 - acc: 0.9693 - val_loss: 0.2388 - val_acc: 0.9785
Running epoch: 1
Epoch 1/1
23000/23000 [==============================] - 648s - loss: 0.3462 - acc: 0.9748 - val_loss: 0.2495 - val_acc: 0.9830
Running epoch: 2
Epoch 1/1
23000/23000 [==============================] - 648s - loss: 0.3082 - acc: 0.9780 - val_loss: 0.3006 - val_acc: 0.9775
Completed 3 fit operations


# Generate Predictions

Now use our new model to make predictions on the test dataset

In [18]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

Found 12500 images belonging to 1 classes.


In [19]:
#For every image, vgg.test() generates two probabilities
#based on how we've ordered the cats/dogs directories
#It looks like column one is cats and column two is dogs
print preds[:5]

filenames = batches.filenames
print filenames[:5]

[[ 0.9839  0.0161]
 [ 0.6823  0.3177]
 [ 0.9222  0.0778]
 [ 0.9987  0.0013]
 [ 0.5726  0.4274]]
['unknown/9292.jpg', 'unknown/12026.jpg', 'unknown/9688.jpg', 'unknown/4392.jpg', 'unknown/779.jpg']


In [20]:
from PIL import Image
Image.open(test_path + filenames[2])

IOError: [Errno 2] No such file or directory: '/home/ubuntu/courses/deeplearning1/nbs/lesson1/data/redux/testunknown/9688.jpg'

In [ ]:
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)

Now we are going to look at the following:
1. A few correct labels at random
2. A few incorrect labels at random
3. The most correct labels of each class (ie those with the highest probability that they are correct)
4. The most incorrect labels of each class (ie those with the highest probability that are incorrect)
5. The most uncertain labels (ie those with the probability closest to 0.5)

In [23]:
latest_weights_filename = DATA_HOME_DIR/results/ft2.h5

NameError: name 'results' is not defined

In [21]:
vgg.model.load_weights(results_path+latest_weights_filename)

NameError: name 'latest_weights_filename' is not defined

In [ ]:
val_batches, probs = vgg.test(valid_path, batch_size = batch_size)